In [1]:
!pip install datasets transformers

In [2]:
from datasets import load_dataset
import os
train_dataset = load_dataset(
    "csv",
    data_files='/kaggle/input/nlp-week-3-summarization-with-language-model/train_shards/train1.csv'
)

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights'],
        num_rows: 20000
    })
})

Test bart model and bart description here

In [4]:
from transformers import pipeline, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
def truncate_text_bart(text, max_length=1000):
    tokens = tokenizer.encode(text, truncation=True, max_length=max_length)
    decoded_seq = tokenizer.decode(tokens, skip_special_tokens=True)
    return decoded_seq 
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
actual_summaries = train_dataset['train']['highlights']
print(len(actual_summaries))
print(actual_summaries[0])

20000
Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has no plans to fritter his cash away .
Radcliffe's earnings from first five Potter films have been held in trust fund .


In [6]:
generated_summaries_bart = []
for i in range(len(train_dataset['train']['article'][:100])):
    truncated_text = truncate_text_bart(train_dataset['train']['article'][i])
    summary = summarizer(truncated_text, max_length=100, do_sample=False)
    generated_summaries_bart.append(summary[0]['summary_text'])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [7]:
!pip install rouge

In [8]:
from rouge import Rouge
# Calculate ROUGE scores
rouge = Rouge()
# print(len(generated_summaries))
scores = rouge.get_scores(generated_summaries_bart, actual_summaries[:100], avg=True)
print(scores)

{'rouge-1': {'r': 0.3847223724797258, 'p': 0.31234298045358605, 'f': 0.3422551402526826}, 'rouge-2': {'r': 0.1538733011905496, 'p': 0.12112095557681701, 'f': 0.1344092657851481}, 'rouge-l': {'r': 0.36092751028601167, 'p': 0.2927588468626526, 'f': 0.3209275913423139}}


In [9]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
def truncate_text_t5(text, max_length=511):
    tokens = tokenizer.encode(text, truncation=True, max_length=max_length)
    decoded_seq = tokenizer.decode(tokens, skip_special_tokens=True)
    return decoded_seq 
generated_summaries_t5 = []
for i in range(len(train_dataset['train']['article'][:100])):
    # truncated_text = truncate_text(train_dataset['train']['article'][i])
    summary = summarizer(train_dataset['train']['article'][i], max_length=100, do_sample=False)
    generated_summaries_t5.append(summary[0]['summary_text'])

Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors


In [11]:
rouge = Rouge()
scores = rouge.get_scores(generated_summaries_t5, actual_summaries[:100], avg=True)
print(scores)

{'rouge-1': {'r': 0.2662940332771142, 'p': 0.2519184973401582, 'f': 0.2552768276056769}, 'rouge-2': {'r': 0.08382083972556688, 'p': 0.07809309455333144, 'f': 0.07912530296647971}, 'rouge-l': {'r': 0.24920874813515695, 'p': 0.2357681606669833, 'f': 0.23883608933113146}}


In [12]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, pipeline
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
device = 'cpu'
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum").to(device)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=device)

def truncate_text_pegasus(text, max_length=512):
    tokens = tokenizer.encode(text, truncation=True, max_length=max_length)
    decoded_seq = tokenizer.decode(tokens, skip_special_tokens=True)
    return decoded_seq

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

Device set to use cpu


In [13]:
generated_summaries_pegasus = []
for i in range(len(train_dataset['train']['article'][:100])):
    truncated_text = truncate_text_pegasus(train_dataset['train']['article'][i])
    summary = summarizer(truncated_text, max_length=300, do_sample=False)
    generated_summaries_pegasus.append(summary[0]['summary_text'])

Your max_length is set to 300, but your input_length is only 284. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=142)
Your max_length is set to 300, but your input_length is only 250. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=125)
Your max_length is set to 300, but your input_length is only 291. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=145)
Your max_length is set to 300, but your input_length is only 220. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1

In [14]:
rouge = Rouge()
scores = rouge.get_scores(generated_summaries_pegasus, actual_summaries[:100], avg=True)
print(scores)

{'rouge-1': {'r': 0.1907595893962172, 'p': 0.32703607357892955, 'f': 0.23242992552962524}, 'rouge-2': {'r': 0.0538560020921727, 'p': 0.10189754434095115, 'f': 0.06838848200872492}, 'rouge-l': {'r': 0.16879176646477295, 'p': 0.29017339931819297, 'f': 0.2058140249291875}}
